In [72]:
import rdflib

Load ontology and example

In [73]:
g = rdflib.Graph()
g.parse("../ontology/yang-library.ttl", format='turtle')
g.parse("../examples/yang-server.ttl", format='turtle')
g.parse("../examples/example.ttl", format='turtle')
print(len(g))

621


Load prefixes

In [ ]:
prefixes = """
PREFIX ndc-lb: <https://w3id.org/yang/library#>
PREFIX ndc-sr: <https://w3id.org/yang/server#>
PREFIX dcterms: <http://purl.org/dc/terms/>
"""

## Step 1: Identifying available datastores

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?datastore ?type
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ndc-sr:YangServer .
        ?datastore rdf:type/rdfs:subClassOf* ndc-sr:Datastore ;
            rdf:type ?type .
    }
""")

for row in q_res:
    print(f"Datastore: {row.datastore} Type: {row.type}")

Datastore: https://network-data-catalog.org/NetconfServer/srl/Datastore/candidate Type: https://w3id.org/yang/server#CandidateDatastore
Datastore: https://network-data-catalog.org/NetconfServer/srl/Datastore/intended Type: https://w3id.org/yang/server#IntendedDatastore
Datastore: https://network-data-catalog.org/NetconfServer/srl/Datastore/running Type: https://w3id.org/yang/server#RunningDatastore
Datastore: https://network-data-catalog.org/NetconfServer/srl/Datastore/startup Type: https://w3id.org/yang/server#StartupDatastore
Datastore: https://network-data-catalog.org/NetconfServer/srl/Datastore/operational Type: https://w3id.org/yang/server#OperationalDatastore


## Step 2: Identifying implemented YANG modules for interface management

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?implementedModule ?dependency
    WHERE {
        ?moduleImplementation ndc-lb:implementedModule ?implementedModule .
        ?moduleDependency rdfs:subPropertyOf+ ndc-lb:moduleDependency .
        ?implementedModule ?moduleDependency ?dependency .
    }
""")

for row in q_res:
    print(f"Module: {row.implementedModule} Dependency: {row.dependency}")

Module: https://network-data-catalog.org/Module/openconfig-interfaces/2023-11-06 Dependency: https://network-data-catalog.org/Module/openconfig-srl-deviations/2024-10-31


## Step : Identifying dependencies for imported YANG modules

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?importedModule ?dependency
    WHERE {
        ?moduleImport ndc-lb:importedModule ?importedModule .
        ?moduleDependency rdfs:subPropertyOf+ ndc-lb:moduleDependency .
        ?importedModule ?moduleDependency ?dependency .
    }
""")

for row in q_res:
    print(f"Module: {row.importedModule} Dependency: {row.dependency}")

Module: https://network-data-catalog.org/Module/openconfig-aft/2023-09-26 Dependency: https://network-data-catalog.org/Module/openconfig-aft-common/2023-09-26
Module: https://network-data-catalog.org/Module/openconfig-aft/2023-09-26 Dependency: https://network-data-catalog.org/Module/openconfig-aft-ethernet/2023-09-26


## Step 4: Collecting MDT protocol information.

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?type ?addressValue ?username ?password ?capabilities
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ndc-sr:YangServer ;
            rdf:type ?type ;
            ndc-sr:socketAddress ?socketAddress ;
            ndc-sr:serverAccount ?serverAccount .
        ?socketAddress observable:addressValue ?addressValue .
        ?serverAccount ndc-sr:username ?username ;
            core:hasFacet ?accountAuthenticationFacet .
        ?accountAuthenticationFacet observable:password ?password .
    }
""")

for row in q_res:
    print(f"Protocol: {row.type} Address: {row.addressValue} Username: {row.username} Password: {row.password}")

Protocol: https://w3id.org/yang/server#NetconfServer Address: 172.100.100.11:830 Username: admin Password: NokiaSrl1!


## Step 4: Collecting NETCONF capabilities

In [ ]:
q_res = g.query(prefixes + """
    SELECT ?capability
    WHERE {
        ?server rdf:type/rdfs:subClassOf* ndc-sr:NetconfServer ;
            ndc-sr:capability ?capability ;
    }
    GROUP BY ?server
""")

for row in q_res:
    print(f"Capability: {row.capability}")

Capability: https://w3id.org/yang/server#YangLibrary1.1Capability
